# Weighed Scaled Pinball Loss (WSPL)

The WSPL, for a single level, is calculated as:  

$$
WSPL = \sum_{series} w_i \cdot SPL = \sum_{series} w_i \cdot \frac{1}{h}\frac{\sum_{days} PL}{S_i}\\
= \sum_{series} w_i \cdot \frac{1}{h}\frac{\sum_{days} \frac{1}{9}\sum_u PL(u)}{S_i}
$$

Where $w_i$ and $S$ are constants per series, computed as described in the M5 Competitors guide.  


The Pinball Loss for a single quantile is calculated as:

$$
PL(u) = \frac{1}{h}\sum_{days}\left[ E \cdot u \text{  or  } E\cdot (1-u)  \right]
$$

Where $E$ is the absolute prediction error: $|Y_t - Q_t(u)|$ and the left or right part is evaluated based on wheter the quantity is over- or underpredicted. 

Normalising a particular series with $c_i$ gives:  

$$
E = |Y_t - Q_t(u)| = c_i \cdot |Y_t/c_i - Q_t(u)/c_i| = c_i \cdot E'
$$

Where $E'$ is the prediction error as calculated with the normalised quantities. 

From this follows that:
$$
PL(u) = \frac{1}{h}\sum_{days}\left[ c_i \cdot E' \cdot u \text{  or  } c_i \cdot E'\cdot (1-u)  \right]\\
= \frac{c_i}{h}\sum_{days}\left[ E' \cdot u \text{  or  }  E'\cdot (1-u)  \right]\\
= c_i \cdot PL(u)'
$$

And subsequently:  

$$
WSPL = \sum_{series} w_i \cdot \frac{1}{h}\frac{\sum_{days} \frac{1}{9}\sum_u c_i \cdot PL(u)'}{S_i}\\
= \sum_{series} w_i \cdot \frac{1}{h}\frac{\sum_{days} c_i PL'}{S_i}\\
$$

Setting $c_i = S_i / w_i$ for each series gives:  

$$
WSPL = \sum_{series} w_i \cdot \frac{1}{h}\frac{S_i / w_i\sum_{days} PL'}{S_i}\\
 = \sum_{series} \frac{1}{h}\sum_{days} PL'\\
$$

So with the chosen normalisation constant, minimizing the 'naive' pinball loss corresponds to the minimization of the weighed scaled pinball loss. 

**Note**: For the values to coincode exactly, the constants $h$ and $\frac{1}{9}$ need to be implemented correctly. This, however, only scales the total loss and does not influence any training decisions. (But could influence magnitude of gradient for all training steps with the same constant)

## Effect of normalisation for higher aggregation levels

Within the M5Forecast - Uncertainty each (aggregated) series must be provided directly, resulting in a total of $42,840$ series (compared to the ~$30k$ product-level series for the M5Forecast - Accuracy challenge).  

For each aggregation level the same computations hold as for the product level pinball loss, but with their respective $w_i$ and $S_i$. 

## Approximate conversion of sales
The weight of each series is calculated using the 28 days preceding the prediction range. To convert the complete dataset in a consistent manner, the weights as calculated using the final 28 days is used. These are available through the `Referee` object. 